<a href="https://colab.research.google.com/github/Vineetiitg/Real-time-Toxic-Chat-Moderation-System/blob/main/Toxic_Chat_Detection_using_RoBERTa(LoRA_PEFT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Toxic_Chat_Detection_using_RoBERTausing LoRA fine Tuning (Active Development)**

In [3]:
!pip install transformers[datasets,evaluate]
!pip install peft

In [4]:
# importing libraries
import pandas as pd
import spacy
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
#from keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df1 = pd.read_csv('/content/drive/MyDrive/train.csv')

In [7]:
df = pd.read_csv('/content/drive/MyDrive/processed_data_for_model.csv')

In [8]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comment_text         7979 non-null   object
 1   toxic                7979 non-null   int64 
 2   severe_toxic         7979 non-null   int64 
 3   obscene              7979 non-null   int64 
 4   threat               7979 non-null   int64 
 5   insult               7979 non-null   int64 
 6   identity_hate        7979 non-null   int64 
 7   processed_comment    7979 non-null   object
 8   processed_comment_2  7979 non-null   object
 9   class                7979 non-null   int64 
 10  lemma_comment        7979 non-null   object
 11  final_comment        7978 non-null   object
dtypes: int64(7), object(5)
memory usage: 748.2+ KB


In [9]:

from sklearn.model_selection import train_test_split


# filter out 20% of the data while class proportions balaced
remaining_df, filtered_df = train_test_split(
    df,
    test_size=0.06,
    stratify=df['class'],
    random_state=42
)

print(f"Original distribution:\n{df['class'].value_counts()}")
print(f"Filtered distribution:\n{filtered_df['class'].value_counts()}")

Original distribution:
class
0    7168
2     677
1     134
Name: count, dtype: int64
Filtered distribution:
class
0    430
2     41
1      8
Name: count, dtype: int64


In [10]:
from transformers import RobertaTokenizerFast
import numpy as np

# 1. Initialize a tokenizer by loading the pre-trained 'roberta-base' tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# 2. Tokenize the df['final_comment'] column
max_length = 128 # Define a suitable maximum length
tokenized_data = tokenizer(list(df['final_comment'].fillna('')), # Fill NaN with empty string
                           truncation=True,
                           padding='max_length',
                           max_length=max_length,
                           return_tensors='np' # Return numpy arrays
                          )

# 3. Extract the input IDs, attention masks
input_ids = tokenized_data['input_ids']
attention_mask = tokenized_data['attention_mask']

labels = df['class'].to_numpy()

print("Tokenization complete.")
print(f"Shape of input_ids: {input_ids.shape}")
print(f"Shape of attention_mask: {attention_mask.shape}")
print(f"Shape of labels: {labels.shape}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenization complete.
Shape of input_ids: (7979, 128)
Shape of attention_mask: (7979, 128)
Shape of labels: (7979,)


## Split Data for RoBERTa Models

In [11]:
from sklearn.model_selection import train_test_split

# Split input_ids, attention_mask, and labels into training and testing sets
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids,
    attention_mask,
    labels,
    test_size=0.2,    # 20% for testing
    random_state=42
)

print("Data split into training and validation sets.")
print(f"Shape of train_input_ids: {train_input_ids.shape}")
print(f"Shape of val_input_ids: {val_input_ids.shape}")
print(f"Shape of train_labels: {train_labels.shape}")
print(f"Shape of val_labels: {val_labels.shape}")

Data split into training and validation sets.
Shape of train_input_ids: (6383, 128)
Shape of val_input_ids: (1596, 128)
Shape of train_labels: (6383,)
Shape of val_labels: (1596,)


In [12]:
import torch
from torch.utils.data import Dataset
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Custom Dataset class for RoBERTa inputs
class HateSpeechDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create training and validation datasets
train_dataset = HateSpeechDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = HateSpeechDataset(val_input_ids, val_attention_mask, val_labels)

print("Custom Dataset classes created and datasets initialized.")

Custom Dataset classes created and datasets initialized.


In [13]:
import numpy as np
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# 3. pre-trained 'roberta-base' model]
num_labels = len(np.unique(labels)) # Automatically determine the number of classes
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# 4. TrainingArguments
training_args = TrainingArguments(
    output_dir='./results_roberta_base',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_roberta_base',
    logging_steps=100,

    report_to="none" # Disable  W&B
)

# 5. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=None,
)

# 6. Call the train() method to start the finetuning process
print("Starting RoBERTa base model training...")
trainer.train()
print("RoBERTa base model training complete.")

# 7.  evaluate the model on the validation set
print("Evaluating RoBERTa base model on validation set...")
eval_results = trainer.evaluate()
print(f"RoBERta Base Model Evaluation Results: {eval_results}")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.out_proj.bias        | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Starting RoBERTa base model training...


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

RoBERTa base model training complete.
Evaluating RoBERTa base model on validation set...


RoBERta Base Model Evaluation Results: {'eval_loss': 0.3936281204223633, 'eval_runtime': 11.1586, 'eval_samples_per_second': 143.029, 'eval_steps_per_second': 8.962, 'epoch': 3.0}


In [14]:
import os

# directory to save the model and tokenizer
roberta_base_model_path = './roberta_base_model'
os.makedirs(roberta_base_model_path, exist_ok=True)

# Save the trained model
model.save_pretrained(roberta_base_model_path)
print(f"RoBERTa base model saved to {roberta_base_model_path}")

# Save the tokenizer
tokenizer.save_pretrained(roberta_base_model_path)
print(f"RoBERTa tokenizer saved to {roberta_base_model_path}")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

RoBERTa base model saved to ./roberta_base_model
RoBERTa tokenizer saved to ./roberta_base_model


In [15]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import numpy as np

num_labels = len(np.unique(labels))
lora_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# 2. Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)


lora_model = get_peft_model(lora_model, lora_config)
print("LoRA-enabled model summary:")
lora_model.print_trainable_parameters()

# 4. Define TrainingArguments for this LoRA finetuning process
training_args_lora = TrainingArguments(
    output_dir='./results_roberta_lora',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_roberta_lora',
    logging_steps=100,
    report_to="none"
)

# 5. Initialize a Trainer with the LoRA-enabled model
trainer_lora = Trainer(
    model=lora_model,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 6. Call the train() method to start the LoRA finetuning process
print("Starting LoRA-enabled RoBERTa model training...")
trainer_lora.train()
print("LoRA-enabled RoBERTa model training complete.")

# 7. After training, evaluate the LoRA-finetuned model on the validation set
print("Evaluating LoRA-enabled RoBERTa model on validation set...")
eval_results_lora = trainer_lora.evaluate()
print(f"LoRA-enabled RoBERTa Model Evaluation Results: {eval_results_lora}")

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.out_proj.bias        | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


LoRA-enabled model summary:
trainable params: 887,811 || all params: 125,535,750 || trainable%: 0.7072
Starting LoRA-enabled RoBERTa model training...


Step,Training Loss
100,1.102999
200,0.612381
300,0.381501
400,0.354503
500,0.349971
600,0.388732
700,0.333896
800,0.383719
900,0.339984
1000,0.357179


LoRA-enabled RoBERTa model training complete.
Evaluating LoRA-enabled RoBERTa model on validation set...


LoRA-enabled RoBERTa Model Evaluation Results: {'eval_loss': 0.35213765501976013, 'eval_runtime': 11.8919, 'eval_samples_per_second': 134.209, 'eval_steps_per_second': 8.409, 'epoch': 3.0}


In [16]:
import os

# directory to save the LoRA-enabled model and tokenizer
roberta_lora_model_path = './roberta_lora_model'
os.makedirs(roberta_lora_model_path, exist_ok=True)

# Save the trained LoRA model
lora_model.save_pretrained(roberta_lora_model_path)
print(f"LoRA-enabled RoBERTa model saved to {roberta_lora_model_path}")

# Save the tokenizer (using the same tokenizer as for the base model, as it's typically the same)
tokenizer.save_pretrained(roberta_lora_model_path)
print(f"RoBERTa tokenizer saved to {roberta_lora_model_path}")

LoRA-enabled RoBERTa model saved to ./roberta_lora_model
RoBERTa tokenizer saved to ./roberta_lora_model


In [17]:
import torch
import torch.nn.functional as F

# Generate predictions (logits) from the trained RoBERTa base model on the validation set
roberta_base_predictions_output = trainer.predict(val_dataset)

# Extract logits
roberta_base_logits = roberta_base_predictions_output.predictions

# Convert logits to probabilities using softmax
roberta_base_predictions_proba = F.softmax(torch.tensor(roberta_base_logits), dim=-1).numpy()

print("RoBERTa base model predictions generated.")
print(f"Shape of RoBERTa base predictions: {roberta_base_predictions_proba.shape}")
print("First 5 predictions (probabilities per class):\n", roberta_base_predictions_proba[:5])

RoBERTa base model predictions generated.
Shape of RoBERTa base predictions: (1596, 3)
First 5 predictions (probabilities per class):
 [[0.9209559  0.00814936 0.07089476]
 [0.9209655  0.00814832 0.07088614]
 [0.92091376 0.00815346 0.07093273]
 [0.92093354 0.00815139 0.07091507]
 [0.9209544  0.00814943 0.07089618]]


In [18]:
import torch
import torch.nn.functional as F

# Generate predictions (logits) from the trained LoRA-enabled RoBERTa model on the validation set
lora_predictions_output = trainer_lora.predict(val_dataset)

# Extract logits
lora_logits = lora_predictions_output.predictions

# Convert logits to probabilities using softmax
lora_predictions_proba = F.softmax(torch.tensor(lora_logits), dim=-1).numpy()

print("LoRA-enabled RoBERTa model predictions generated.")
print(f"Shape of LoRA predictions: {lora_predictions_proba.shape}")
print("First 5 predictions (probabilities per class):\n", lora_predictions_proba[:5])

LoRA-enabled RoBERTa model predictions generated.
Shape of LoRA predictions: (1596, 3)
First 5 predictions (probabilities per class):
 [[0.9556878  0.00382498 0.04048719]
 [0.9509276  0.00421215 0.04486023]
 [0.7821457  0.02163996 0.19621436]
 [0.9679176  0.00295117 0.02913119]
 [0.9373031  0.00489399 0.05780287]]
